# Syllabys

1. [Why containerization is needed](#why-containerization-is-needed)
2. [Docker overview](#docker-overview)
3. [Image syntax overview](#image-syntax-overview)
4. [Commands overview](#commands-overview)
5. [CUDA Containers](#cuda-containers)
6. [Docker Compose](#docker-compose)
7. [Homework](#homework)

![Docker architecture example](https://media.makeameme.org/created/it-worked-on-d9e3629876.jpg)

# Why containerization is needed

Imagine a situation in which you need to develop a CV project.

As a developer usually you'll start experimenting with the development on your local environment over which you have most control.

You test different libraries, models, etc. Then you work on deployment - creating an Web API interface for other people to use. And in the end you need to `share the project with steps of how to reproduce the whole working project on any host`. 

What would steps be in abstract form?

1. Installing Python
2. Installing middleware (OpenCV needs special C libraries, most Deep Learning models need GPU)
3. Installing libraries used in the project
4. Running a project with specific configuration (need specific arguments or can be made as chain of scripts)

In abstract form it looks easy enough but `in details usually it's around 20-30 CLI commands that are needed to be run in specific order`.

And another important thing is a `platform` on which solution is going to be deployed.

Instructions from above can differ depending on the platform.

## Docker vs VM (Virtual Machine)

![Docker architecture example](https://k21academy.com/wp-content/uploads/2020/11/Docker-and-Vm-blog-image_result-1.webp)

# Docker overview

`Docker` is a platform that allows to develop, deploy and run applications with `containers`.

`Container` is an isolated environment from your infrastructure. This means that a container has no knowledge of your operating system, it's middleware, or your files. It runs on the environment provided to you by Docker. Containers have everything that your code needs in order to run, down to a base operating system.

![Docker architecture example](https://code.visualstudio.com/assets/learn/develop-cloud/containers/container-architecture.png)

Docker pros:

- `Consistency`: Docker ensures consistent environments across development, testing, and production, eliminating "it works on my machine" issues.
- `Isolation`: Containers encapsulate applications and their dependencies, preventing conflicts and ensuring reproducibility.
- `Portability`: Docker containers can run on any platform that supports Docker, enabling seamless deployment across diverse environments.
- `Efficiency`: Docker's lightweight nature allows for faster deployment, scaling, and resource utilization compared to traditional virtual machines.


Docker difference from other isolation techniques:

1. Virtual environment - it adds isolation on the code interpreter level but doesn't aim at middleware isolation (meaning that system libraries and drivers cannot be isolated by it).
2. Virtual machine - it follows completely different architecture that uses `hypervisor` - a middleware that isolates operating systems on top of infrastructure. Docker does not fully emulate operating system, it uses more light-weighted approach that uses Host Operating System.

## Docker Architecture

![](https://docs.docker.com/get-started/images/docker-architecture.webp)

Docker uses a `client-server architecture`. The Docker client talks to the Docker daemon, which does the heavy lifting of building, running, and distributing your Docker containers. The Docker client and daemon can run on the same system, or you can connect a Docker client to a remote Docker daemon. The Docker client and daemon communicate using a REST API.

- `The Docker daemon`

    The Docker daemon (dockerd) listens for Docker API requests and `manages Docker objects` such as images, containers, networks, and volumes. A daemon can also communicate with other daemons to manage Docker services.

- `The Docker client`

    The Docker client (docker) is the primary way that many Docker users interact with Docker. When you use commands such as `docker run`, the `client sends these commands to dockerd, which carries them out`. The docker command uses the Docker API. The Docker client can communicate with more than one daemon.

- `Docker registries`

    A Docker registry `stores Docker images`. Docker Hub is a public registry that anyone can use, and Docker looks for images on Docker Hub by default. You can even run your own private registry.

    When you use the `docker pull` or `docker run` commands, Docker pulls the required images from your configured registry. When you use the `docker push` command, Docker pushes your image to your configured registry.

- `Docker Images`

    An image is a `read-only template with instructions for creating a Docker container`. Often, an image is based on another image, with some additional customization. For example, you may build an image which is based on the ubuntu image, but installs the Apache web server and your application, as well as the configuration details needed to make your application run.

    You might create your own images or you might only use those created by others and published in a registry. To build your own image, you create a Dockerfile with a simple syntax for defining the steps needed to create the image and run it. Each `instruction in a Dockerfile creates a layer` in the image. When you change the Dockerfile and rebuild the image, only those `layers which have changed are rebuilt`. This is part of what makes images so lightweight, small, and fast, when compared to other virtualization technologies.

- `Docker Containers`

    A container is a `runnable instance of an image`. You can create, start, stop, move, or delete a container using the Docker API or CLI. You can connect a container to one or more networks, attach storage to it, or even create a new image based on its current state.

    By default, a container is relatively well isolated from other containers and its host machine. You can control how isolated a container's network, storage, or other underlying subsystems are from other containers or from the host machine.

    A container is defined by its image as well as any configuration options you provide to it when you create or start it. When a container is removed, any changes to its state that aren't stored in persistent storage disappear.


[How to install Docker](https://docs.docker.com/get-docker/) described here for each platform available.

# Image syntax overview

An image is a `read-only template with instructions for creating a Docker container`. 

To build your own image, you create a `Dockerfile with a simple syntax for defining the steps needed to create the image and run it`. 

Each `instruction in a Dockerfile creates a layer` in the image. 

When you change the Dockerfile and rebuild the image, only those `layers which have changed are rebuilt`. This is part of what makes images so lightweight, small, and fast, when compared to other virtualization technologies.


OOP analogy:

- image = class
- layer = inheritance
- container = object

Docker instruction are described in `Dockerfile`.


Example of base Dockerfile:

```docker
# base image
FROM python:3.6

# specify working directory
WORKDIR /app

# OR copy all files into workdir
COPY . .

# default command to be executed on container start
CMD python app.py
```

Let's break this commands:

- `FROM` - sets a base image (image from Docker Hub that has predefined instructions)
- `WORKDIR`- sets a working directory for any RUN, CMD, ENTRYPOINT, COPY and ADD instructions that follow it in the Dockerfile
- `COPY` - copies files from specified filesystem into isolated container's filesystem
    * `.dockerignore` file can be used to ignore files by specific included pattern
- `ENV` - sets an environment variable for a container
- `EXPOSE` - informs Docker that the container listens on the specified network ports at runtime
    * on a runtine port has to be published using `-p [internal port]:[external port]` arguments
- `ENTRYPOINT` sets the main command that always runs 
- `RUN` - executes command as a new layer to be set in a container
- `CMD` - sets the command to be executed when running a container from an image



<h3>Base images (FROM command)</h3>

Base images defines initial setup of container's environment. 

They can be found in [Docker Hub](https://hub.docker.com/search?image_filter=official&q=).

Each image also can have a tag - symbolic representation of version or specific configuration. 
The format of image name is `{name}:{tag}`, by default tag is `latest`.


Docker Hub contains next type of images:

- operating system (such as `alpine`, `ubuntu`, `debian`, etc., usually open-sourced OS)
- platforms, frameworks or systems (`kafka`, `postgres`, `python`, etc.) 

### Difference between ENTRYPOINT and CMD

```docker run myimage``` -> ```python app.py``` <br>
```docker run myimage other_script.py``` -> ```python other_script.py```

Example of base Dockerfile:

```docker
# base image
FROM python:3.6

# specify working directory
...

ENTRYPOINT ["python"]
CMD ["app.py"]
```

Example of Dockerfile for FastAPI [is listed here](./fast_api_demo/Dockerfile).

# Commands overview

After creating an `image` with Dockerfile, command `docker build` allows to build a container based on supplied image.

Command format:

```bash
docker build -t {image name} -f {filename} PATH
```
where:
- `PATH` is a required argument that specifies the build context
- `-t {name:tag}` - sets name of the image with the tag
- `f {filename}` - specifies the name of the file that contains build instructions, by default Dockerfile


After building a Docker Image, it can be started as a `Container` using command `docker run`.

Command format:

```bash
docker run {OPTIONS} IMAGE_NAME
```
where:
- `IMAGE_NAME` is a required name of the image that has to be run
- `-d` - detaches process running the container in background
- `-it` - contains two commands: `i` for interactive mode that allows sending input to the container and `t` for allocating terminal for the communication with the container
- `-p {internal port}:{external port}` - connects container port with external port for communication
- `--rm` - flag for cleaning container states (removing previous states saved in the memory)
- `--name {NAME}` - assigns custom identifier for a container
- `--gpus {all|devices=NAMES}` - allows to access NVIDIA GPU resources, more about enabling GPU [is written here](#cuda-containers)
- `-e {VAR=VAL}` - sets environment variable
- `--restart {no|always|unless-stoped|on-failure}` - controlls restart policy forf a container after it's exit
- `-v {external directory}:{internal directory}` - mounts directory into container and allows file sharing

List of other useful commands:

- `docker images` - shows all installed images
- `docker rmi {IMAGE_NAME}` - deletes image
- `docker ps -a` - shows all containers (without `-a` flag shows only ones that are running)
- `docker stop {CONTAINER_NAME_OR_ID}` - stops docker container
- `docker rm {CONTAINER_NAME_OR_ID}` - removes docker container
- `docker logs {CONTAINER_NAME_OR_ID}` - shows container logs
- `docker exec {CONTAINER_NAME_OR_ID} {COMMAND}` - executes a command in a running container, usually used as:
    * `docker exec -it {CONTAINER_NAME_OR_ID} bash` to start a bash session and connect to existing
- [and many other commands listed here](https://docs.docker.com/reference/cli/docker/)

### Caching

In [18]:
!docker build -t fasapi_example:dev -f fast_api_demo/Dockerfile fast_api_demo/

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 589B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.10             0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 589B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.10             0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 589B                                       0.0s
 => [internal] load metadata for docker.

In [19]:
!docker run -it --rm -p 5000:5000 fasapi_example:dev

INFO:     Will watch for changes in these directories: ['/app']
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)
INFO:     Started reloader process [1] using StatReload
122
INFO:     Started server process [8]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     172.17.0.1:35698 - "GET / HTTP/1.1" 200 OK
INFO:     172.17.0.1:35706 - "GET /docs HTTP/1.1" 200 OK
INFO:     172.17.0.1:35706 - "GET /openapi.json HTTP/1.1" 200 OK
^C
INFO:     Shutting down
INFO:     Finished server process [8]
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/starlette/routing.py", line 747, in lifespan
    await receive()
  File "/usr/local/lib/python3.10/site-packages/uvicorn/lifespan/on.py", line 139, in receive
    return await self.receive_queue.get()
  File "/usr/local/lib/python3.10/asyncio/queues.py", line 159, in get
    await getter
asyncio.exceptions.CancelledError



# CUDA Containers

By default, GPU-acceleration is not available in containers. To enable it, install [NVIDIA Container toolkit](https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/latest/install-guide.html).

After that GPUs can be enabled in `docker run` via `--gpu {all}` option.

You can use next CUDA images from Docker Hub:

- `nvidia/cuda`
- `pytorch/pytorch` with tag for CUDA and CuDNN such as `:2.0.1-cuda11.7-cudnn8-runtime`
- `jupyter/pytorch-notebook` with tag for CUDA such as `:cuda11-latest`
- other images with checking their tags.



Example of Dockerfile for FastAPI builing on base CUDA image [is listed here](./fast_api_demo/Dockerfile_gpu).

<h3>Building consistent working environment</h3>

One of the usages of Containers is creating isolated working environment in case when you need to experiment on different middleware (CUDA, different drivers for specific packages, etc.).

Example of setuping a Docker container with PyTorch, CUDA and Jupyter Notebook for experimentation [is listed here](./Dockerfile_gpu_env).

In [5]:
# Build a docker image
!docker build -t gpu_example -f fast_api_demo/Dockerfile_gpu fast_api_demo/

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile_gpu                   0.0s
 => => transferring dockerfile: 533B                                       0.0s
 => [internal] load metadata for docker.io/nvidia/cuda:11.8.0-base-ubuntu  0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile_gpu                   0.0s
 => => transferring dockerfile: 533B                                       0.0s
 => [internal] load metadata for docker.io/nvidia/cuda:11.8.0-base-ubuntu  0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile_gpu                   0.0s
 => => transferring dockerfile: 533B                                       0.0s
 => [internal] load metadata for docker.

In [23]:
!docker run --gpus all gpu_example:latest

True
NVIDIA GeForce RTX 3050 Laptop GPU


In [21]:
# View available images
!docker images

REPOSITORY         TAG              IMAGE ID       CREATED          SIZE
fasapi_example     dev              b7e91ab1a470   2 minutes ago    7.34GB
<none>             <none>           d82e2494e3de   3 minutes ago    7.34GB
<none>             <none>           3f581c8b51c8   26 minutes ago   7.34GB
<none>             <none>           249293bf9689   2 hours ago      7.34GB
<none>             <none>           3b86e2b5f886   2 hours ago      7.34GB
<none>             <none>           ab963c1fe9a2   4 hours ago      6.15GB
<none>             <none>           bf10ce82e6a2   4 hours ago      6.15GB
<none>             <none>           d0515ea7a464   4 hours ago      6.15GB
<none>             <none>           2221aa3c0e92   4 hours ago      6.15GB
gpu_example        latest           baba9c077dc4   4 hours ago      6.15GB
<none>             <none>           e1520c155bac   4 hours ago      6.15GB
<none>             <none>           af75688a1bdb   4 hours ago      6.15GB
<none>             <none>  

In [ ]:
# Removes dangling images
!dicker images prune

In [22]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [13]:
!docker logs 3f16c7291abd20

[I 2024-03-12 16:49:14.814 ServerApp] jupyter_lsp | extension was successfully linked.
[I 2024-03-12 16:49:14.817 ServerApp] jupyter_server_terminals | extension was successfully linked.
[I 2024-03-12 16:49:14.820 ServerApp] jupyterlab | extension was successfully linked.
[I 2024-03-12 16:49:14.823 ServerApp] notebook | extension was successfully linked.
[I 2024-03-12 16:49:14.824 ServerApp] Writing Jupyter server cookie secret to /root/.local/share/jupyter/runtime/jupyter_cookie_secret
[I 2024-03-12 16:49:15.001 ServerApp] notebook_shim | extension was successfully linked.
[I 2024-03-12 16:49:15.014 ServerApp] notebook_shim | extension was successfully loaded.
[I 2024-03-12 16:49:15.017 ServerApp] jupyter_lsp | extension was successfully loaded.
[I 2024-03-12 16:49:15.018 ServerApp] jupyter_server_terminals | extension was successfully loaded.
[I 2024-03-12 16:49:15.019 LabApp] JupyterLab extension loaded from /opt/conda/lib/python3.10/site-packages/jupyterlab
[I 2024-03-12 16:49:15.0

In [ ]:
# docker kill as an alternative
!docker stop 38b872dc650f

38b872dc650f


In [2]:
!docker ps

CONTAINER ID   IMAGE                   COMMAND                  CREATED          STATUS          PORTS                                                 NAMES
8f445b282027   ubuntu/kafka            "entrypoint.sh /etc/…"   19 seconds ago   Up 17 seconds                                                         lesson_36_docker-kafka-1
5b0f28f9abf9   ubuntu/zookeeper:edge   "/opt/kafka/bin/zook…"   19 seconds ago   Up 17 seconds                                                         lesson_36_docker-zookeeper-1
3f16c7291abd   work-env                "jupyter notebook --…"   10 minutes ago   Up 10 minutes   0.0.0.0:8887->8888/tcp, :::8887->8888/tcp             musing_benz
21b4c842ee53   work-env                "jupyter notebook --…"   11 minutes ago   Up 11 minutes   8888/tcp, 0.0.0.0:8889->8889/tcp, :::8889->8889/tcp   goofy_kirch
38b872dc650f   work-env                "jupyter notebook --…"   16 minutes ago   Up 16 minutes   0.0.0.0:8888->8888/tcp, :::8888->8888/tcp             zen_carver

# Docker Compose

Docker Compose is a tool for defining and running multi-container applications.

Compose simplifies the control of your entire application stack, making it easy to manage services, networks, and volumes in a single, comprehensible YAML configuration file. Then, with a single command, you create and start all the services from your configuration file.

![Docker Compose Architecture example](https://www.biaudelle.fr/wp-content/uploads/2021/07/docker-compose-archi.png)



Instructions on how to install Docker Compose [are located here](https://docs.docker.com/compose/install/).

Docker Compose uses `compose.yaml` (or `docker-compose.yaml`, `docker-compose.yml`) files that are placed in the working directory and contains Compose instructions.

<h3>Format of Compose Specifications</h3>

1. At the top level next elements are defined - `version`, `name`, `services`, `networks`, `volumes`, `configs` and `secrets`.

    The most important element is `services` as they define containers that has to be built by Docker Compose.

2. Inside `services` block each element is a name of a service inside which each section defines configuration of that service:

    - `image` - contains a name of Docker image
    - `environment` - section that contains environment variables
    - `ports` - section that contains list of internal and external ports to be mapped for a service
    - `network_mode` - sets a service container's network mode
    - `depends_on` - expresses startup and shutdown dependencies between services
    - `build` - section that defines the build process, it's format [is listed here](https://docs.docker.com/compose/compose-file/build/#illustrative-example)
    - `command` - overrides the default command declared by the container image
    - `volumes` - defines volumes that are accessible by servce containers
    - `deploy` - section that defines metadata for a service (such as it's resources, limitations, restart policies, etc.), it's format [is listed here](https://docs.docker.com/compose/compose-file/deploy/)
    - and many other that [are listed here](https://docs.docker.com/compose/compose-file/05-services/)

Example of Docker Compose for a Kafka and Jupyter Lab env [is listed here](./docker-compose.yml).

In [24]:
!docker compose up

WARN[0000] /home/x_lts/Desktop/CAMP/camp_2025/lesson_36_docker/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
[+] Running 2/2
 ✔ Container lesson_36_docker-zookeeper-1  Created                         0.0s 
 ✔ Container lesson_36_docker-kafka-1      Created                         0.0s 
Attaching to kafka-1, zookeeper-1
78zookeeper-1  | [2025-06-27 14:53:17,358] INFO Reading configuration from: /etc/kafka/zookeeper.properties (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
78

7w Enable Watch878zookeeper-1  | [2025-06-27 14:53:17,365] INFO clientPortAddress is 0.0.0.0:2181 (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
78

7w Enable Watch878zookeeper-1  | [2025-06-27 14:53:17,365] INFO secureClientPort is not set (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
78

7w Enable Watch878zookeeper-1  | [2025-06-27 14:53:17,365] INFO observerMasterPort is not set (org.apache.zoo

In [ ]:
!docker compose down --volumes

# Homework

1. Create a Docker deployment based on some previous homework of your choosing (preferably some WebAPI solution). 

2. Make it so u could run any script inside the container by providing some argumnets to 'docker run' command (hint. check ENTRYPOINT and CMD).

3. When submitting homeworks, add dockerfile as well as screenshots that fully showcase each step of the process and the running container (some screenshot of the deployed webpage and logs from the container).